In [1]:
# File: enhanced_stock_trading_strategy_v4_debug_fixed.py

import alpaca_trade_api as tradeapi
import pandas as pd
from datetime import datetime, timedelta
from config import API_KEY, SECRET_KEY, BASE_URL

# Initialize Alpaca API
api = tradeapi.REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')

# Expanded stock and ETF symbols list
symbols = [
    # Mega-cap tech
    'AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'TSLA', 'NVDA', 'AMD',

    # Large-cap growth and momentum stocks
    'NFLX', 'SHOP', 'SQ', 'PYPL', 'ROKU', 'SPOT', 'UBER', 'LYFT', 'ZM', 'BABA',

    # Semiconductor and tech equipment
    'ASML', 'TXN', 'QCOM', 'AVGO', 'INTC', 'MU', 'LRCX', 'MRVL',

    # Fintech and digital payment companies
    'V', 'MA', 'AXP', 'INTU', 'COIN', 'AFRM', 'SOFI', 'FISV',

    # Consumer discretionary and e-commerce
    'HD', 'LOW', 'COST', 'WMT', 'TGT', 'NKE', 'LULU', 'SBUX', 'MCD',

    # Industrial and infrastructure
    'CAT', 'DE', 'MMM', 'UNP', 'BA', 'GE', 'HON',

    # Energy and alternative energy
    'XOM', 'CVX', 'SLB', 'PLUG', 'RUN', 'ENPH', 'FSLR', 'SPWR',

    # Healthcare and biotechnology
    'JNJ', 'PFE', 'MRK', 'LLY', 'ABBV', 'BIIB', 'GILD', 'REGN', 'VRTX', 'BNTX',

    # Retail and e-commerce
    'AMZN', 'WMT', 'EBAY', 'ETSY', 'CHWY', 'JD', 'MELI',

    # Financials and banks
    'JPM', 'GS', 'MS', 'BAC', 'C', 'WFC', 'BLK', 'SCHW', 'TROW',

    # Telecom and media
    'T', 'VZ', 'DIS', 'CMCSA', 'TMUS', 'NFLX', 'DISCA', 'PARA',

    # Precious metals and materials
    'NEM', 'FCX', 'CLF', 'VALE', 'BHP', 'RIO',

    # ETFs (growth, tech, semiconductors, leveraged ETFs)
    'TQQQ', 'SOXL', 'SPY', 'QQQ', 'DIA', 'ARKK', 'ARKG', 'ARKF', 'SMH', 'XLF', 'XLE',

    # International large caps
    'BABA', 'TCEHY', 'NIO', 'TSM', 'INFY', 'SAP', 'RY', 'TD'
]

# Define ETFs subset
etfs = {'TQQQ', 'SOXL', 'SPY', 'QQQ', 'DIA', 'ARKK', 'ARKG', 'ARKF', 'SMH', 'XLF', 'XLE'}

# Define date range and settings
timeframe = tradeapi.TimeFrame.Day
start_date = '2023-08-19'
end_date = '2024-08-19'
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=50)
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

# Fetch stock data
bars = api.get_bars(
    symbols, timeframe,
    start=start_date_dt.strftime('%Y-%m-%d'),
    end=end_date_dt.strftime('%Y-%m-%d')
).df.sort_index()

# Detect drop days with adjusted thresholds
def detect_drop_days(bars, drop_threshold_low=-6, drop_threshold_high=-1):
    bars['percent_change'] = bars['close'].pct_change() * 100
    drop_days = bars[(bars['percent_change'] <= drop_threshold_high) & (bars['percent_change'] >= drop_threshold_low)]
    print(f"Detected {len(drop_days)} drop days")  # Debugging line
    return drop_days

# Refined trading strategy with relaxed entry criteria
def apply_trading_strategy(bars, drop_days, max_allocation_per_trade=0.20, stop_loss_threshold=-2.0, take_profit_threshold=2.5, etf_trailing_stop=2.5):
    initial_balance = 100000
    balance = initial_balance
    open_positions = []
    trade_log = []

    print(f"Starting balance: ${initial_balance}")

    for index, drop_day in drop_days.iterrows():
        symbol = drop_day['symbol']
        drop_date = index
        symbol_bars = bars[bars['symbol'] == symbol]
        post_drop_data = symbol_bars.loc[drop_date:]

        # Entry based on any green day after a drop
        if len(post_drop_data) > 1:
            day_1 = post_drop_data.iloc[1]
            if day_1['close'] > day_1['open']:
                entry_price = day_1['open']
                available_cash = balance * max_allocation_per_trade
                shares_to_buy = available_cash // entry_price
                trade_value = shares_to_buy * entry_price

                if trade_value <= balance:
                    trailing_stop = etf_trailing_stop if symbol in etfs else stop_loss_threshold
                    open_positions.append({
                        'symbol': symbol,
                        'entry_price': entry_price,
                        'shares': shares_to_buy,
                        'initial_date': drop_date,
                        'trade_value': trade_value,
                        'trailing_stop_price': entry_price * (1 - trailing_stop / 100),
                        'take_profit_price': entry_price * (1 + take_profit_threshold / 100),
                        'has_reentered': False
                    })
                    balance -= trade_value
                    print(f"Entered {symbol} at {entry_price} on {drop_date} with {shares_to_buy} shares, "
                          f"remaining balance: ${balance:.2f}")

        # Manage open positions
        for day in range(2, len(post_drop_data)):
            day_data = post_drop_data.iloc[day]
            current_price = day_data['close']
            for position in open_positions[:]:
                if position['symbol'] == symbol:
                    profit_loss_pct = ((current_price - position['entry_price']) / position['entry_price']) * 100

                    # Trailing stop adjustment
                    if current_price > position['entry_price']:
                        new_trailing_stop = current_price * (1 - (etf_trailing_stop if symbol in etfs else stop_loss_threshold) / 100)
                        position['trailing_stop_price'] = max(position['trailing_stop_price'], new_trailing_stop)
                        print(f"Updated trailing stop for {symbol} to ${position['trailing_stop_price']:.2f}")

                    # Exit condition check
                    if current_price <= position['trailing_stop_price']:
                        balance += current_price * position['shares']
                        trade_log.append({
                            'symbol': symbol,
                            'entry_date': position['initial_date'],
                            'exit_date': day_data.name,
                            'profit_loss_pct': profit_loss_pct,
                            'exit_reason': 'Trailing Stop or Stop-Loss Hit'
                        })
                        open_positions.remove(position)
                        print(f"Exited {symbol} at {current_price} on {day_data.name} due to trailing stop, "
                              f"new balance: ${balance:.2f}")

                    elif current_price >= position['take_profit_price']:
                        balance += current_price * position['shares']
                        trade_log.append({
                            'symbol': symbol,
                            'entry_date': position['initial_date'],
                            'exit_date': day_data.name,
                            'profit_loss_pct': profit_loss_pct,
                            'exit_reason': 'Take Profit Hit'
                        })
                        open_positions.remove(position)
                        print(f"Exited {symbol} at {current_price} on {day_data.name} due to take profit, "
                              f"new balance: ${balance:.2f}")

                    elif day == 3:
                        balance += current_price * position['shares']
                        trade_log.append({
                            'symbol': symbol,
                            'entry_date': position['initial_date'],
                            'exit_date': day_data.name,
                            'profit_loss_pct': profit_loss_pct,
                            'exit_reason': 'Day 3 Exit'
                        })
                        open_positions.remove(position)
                        print(f"Exited {symbol} at {current_price} on {day_data.name} due to Day 3 Exit, "
                              f"new balance: ${balance:.2f}")

    final_balance = balance + sum(p['shares'] * post_drop_data.iloc[-1]['close'] for p in open_positions)
    return_percentage = ((final_balance - initial_balance) / initial_balance) * 100

    print(f"Final portfolio value: ${final_balance:.2f}")
    print(f"Return percentage: {return_percentage:.2f}%")
    print("Trade log:")
    for trade in trade_log:
        print(trade)

# Detect drop days
drop_days = detect_drop_days(bars)

# Apply the refined trading strategy
apply_trading_strategy(bars, drop_days)


Detected 390 drop days
Starting balance: $100000
Entered BABA at 83.98 on 2023-07-03 04:00:00+00:00 with 238.0 shares, remaining balance: $80012.76
Exited BABA at 83.84 on 2023-07-06 04:00:00+00:00 due to trailing stop, new balance: $99966.68
Entered TQQQ at 41.83 on 2023-07-11 04:00:00+00:00 with 477.0 shares, remaining balance: $80013.77
Updated trailing stop for TQQQ to $42.96
Exited TQQQ at 44.06 on 2023-07-13 04:00:00+00:00 due to take profit, new balance: $101030.39
Entered JPM at 148.7 on 2023-07-12 04:00:00+00:00 with 135.0 shares, remaining balance: $80955.89
Updated trailing stop for JPM to $152.77
Exited JPM at 149.77 on 2023-07-14 04:00:00+00:00 due to trailing stop, new balance: $101174.84
Entered AXP at 172.94 on 2023-07-14 04:00:00+00:00 with 117.0 shares, remaining balance: $80940.86
Updated trailing stop for AXP to $181.60
Exited AXP at 178.04 on 2023-07-18 04:00:00+00:00 due to trailing stop, new balance: $101771.54
Entered JPM at 150.35 on 2023-07-14 04:00:00+00:00 w